In [5]:
import os
from collections import defaultdict
from surprise import Dataset, Reader, KNNBasic, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


# 解释代码
# 数据加载：使用Surprise内置的数据集ml-100k，这是一个包含10万条电影评分记录的标准数据集。
# 数据划分：将数据划分为训练集和测试集。
# 算法选择：使用基于物品的协同过滤算法（KNNBasic），相似度度量采用余弦相似度。
# 训练模型：在训练集上训练模型。
# 预测评分：在测试集上进行预测，并计算RMSE（均方根误差）评估模型性能。
# 推荐电影：为每个用户推荐评分最高的前N部电影。
# 这样，你可以利用Surprise库方便地实现和评估推荐系统，而无需从头编写算法逻辑。如果需要处理自己的数据集，可以使用Reader类加载数据，格式如下：

def get_top_n_recommendations(predictions, n=10):
    """为每个用户推荐前N个电影"""
    # 将预测结果整理成{uid: [(iid, est), ...]}的形式
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # 对每个用户的预测评分进行排序，并返回前N个结果
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n


def get_user_recommendations(predictions, user_id, n=10):
    """为指定用户推荐前N个电影"""
    # 将预测结果整理成{uid: [(iid, est), ...]}的形式
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        if uid == user_id:
            top_n[uid].append((iid, est))
    # uid: 用户ID
    # iid: 产品ID（或物品ID）
    # true_r: 实际评分（如果存在的话，用于评估模型的准确性）
    # est: 预测评分（模型预测的评分值）
    # _: 其他信息（通常是一个占位符，可能包括时间戳或其他元数据）
    
    # 对每个用户的预测评分进行排序，并返回前N个结果
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [6]:
# 加载内置的MovieLens 100k数据集
# data = Dataset.load_builtin('ml-100k')

# 读取自己的数据集
# 定义文件格式：line_format='user item rating timestamp' 是 Surprise 库中 Reader 类的参数之一，
# 用于定义数据文件中每一行的格式。这意味着数据文件中的每一行包含四个字段，依次为 user（用户ID）、item（物品ID）、rating（评分）和 timestamp（时间戳）。
# 这些字段的顺序和名称告诉 Surprise 如何解析数据文件中的每一行。
reader = Reader(line_format='user item rating', sep='::')
# 从文件加载数据
# rating_file = os.path.join('file', 'output.dat')
rating_file = '../file/output2.dat'
print(rating_file)
data = Dataset.load_from_file(rating_file, reader=reader)

# 划分训练集和测试集
trainset, testset = train_test_split(data, test_size=0.25)

# >>>>>>计算物品相似度-s
# 基于物品的协同过滤算法
sim_options = {
    'name': 'cosine',  # 使用余弦相似度
    'user_based': False  # False表示基于物品的协同过滤
}
# KNNWithMeans：这个算法在计算相似度时，会考虑评分的平均值。也就是说，它会减去用户或物品的平均评分，然后再计算相似度。这有助于减少评分的偏差，从而使得推荐更加准确。
# KNNBasic：这是最基本的KNN算法，直接计算用户或物品之间的相似度，而不考虑评分的平均值。这种方法可能会受到评分偏差的影响。
# algo = KNNBasic(sim_options=sim_options)

# 使用 SVD 算法，n_factors 潜在特征的数目
algo = SVD(n_factors=12)

# 训练模型
algo.fit(trainset)
# >>>>>>计算物品相似度-e

# 在测试集上进行预测，预测的结果也可以用于实际推荐。
# testset 通常是从数据集中分割出的测试集，包含用户-物品对和对应的实际评分。它用于比较模型的预测评分和真实评分，以计算误差（如 RMSE）。
predictions = algo.test(testset)

# >>>>>>评估模型-s
# 评估模型的准确性
accuracy.rmse(predictions)

# 在训练集上进行预测
# trainset.build_testset() 用于生成一个与训练集结构相同但评分未知的测试集。它主要用于生成模型预测的输入数据。
# 生成的测试集是一个包含所有用户-物品对的列表，其中每个对的评分字段为 None 或者一个占位符值。
# 这些数据对通常用来生成预测评分，而不是用来评估模型性能。
train_predictions = algo.test(trainset.build_testset())

../file/output2.dat
RMSE: 1.0037


In [10]:
# 指定用户ID
# 指定用户的推荐结果
specified_user_id = '10828263'
top_n = get_user_recommendations(train_predictions, specified_user_id, n=10)

# >>>>>>推荐算法-s
# 获取推荐结果
# top_n = get_top_n_recommendations(predictions, n=2)
# top_n = get_top_n_recommendations(train_predictions, n=2)

# 打印推荐结果
for uid, user_ratings in top_n.items():
    print(f"User ——> {uid}:")
    for (iid, est) in user_ratings:
        print(f"  推荐 {iid}: 预测评分 {est}")


User——> 10828263:
  推荐 16515: 预测评分 2.8320215306301595
  推荐 15623: 预测评分 1.8346303439641032


In [11]:
train_predictions

[Prediction(uid='11301388', iid='37236', r_ui=4.12, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='21372', r_ui=6.09, est=4.825170605217448, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='19676', r_ui=6.09, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='17232', r_ui=6.09, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='15446', r_ui=6.09, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='11773', r_ui=6.09, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='15150', r_ui=6.09, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='35092', r_ui=6.09, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='39889', r_ui=6.09, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='34068', r_ui=6.09, est=5, details={'was_impossible': False}),
 Prediction(uid='11301388', iid='2

In [12]:
for uid, iid, true_r, est, _ in predictions:
    if uid == specified_user_id:
        print(uid, iid, true_r, est)

10828263 16330 1.59 2.972992877549823
10828263 15150 1.59 3.0290020786491887
